In [1]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import dash_bootstrap_components as dbc

In [2]:
import pandas as pd
import math

# Read data from Excel files
staff_data = pd.read_excel('staff.xlsx')  # Replace 'path_to_staff.xlsx' with the actual path
types_data = pd.read_excel('types.xlsx')  # Replace 'path_to_types.xlsx' with the actual path

# Merge the data based on the 'Project' column
merged_data = staff_data.merge(types_data, on='Project', how='inner')

# Calculate maximum job number for each job in a month for each project
max_job_numbers = merged_data.groupby(['Type', 'Job', 'Project']).agg({'Month': 'count'}).reset_index()
max_job_numbers = max_job_numbers.rename(columns={'Month': 'Number_of_Jobs'})

# Calculate average maximum job number rounded down
average_max_job_number = math.floor(max_job_numbers['Number_of_Jobs'].mean())

# Calculate average start and end months for each job title in a project
avg_start_end = staff_data.groupby(['Job', 'Project']).agg({'Month': ['min', 'max']}).reset_index()
avg_start_end.columns = ['Job', 'Project', 'Start_Month', 'End_Month']

# Create the new project using the calculated averages
new_project = {
    'Job_Title': [],
    'Start_Month': [],
    'End_Month': [],
    'Number_of_Jobs': []
}

for index, row in avg_start_end.iterrows():
    new_project['Job_Title'].append(row['Job'])
    new_project['Start_Month'].append(row['Start_Month'].month)
    new_project['End_Month'].append(row['End_Month'].month)
    new_project['Number_of_Jobs'].append(average_max_job_number)  # Using the calculated average max job number


# Create final_df DataFrame
final_df = pd.DataFrame(new_project)


In [3]:
import pandas as pd

# Read data from Excel files
staff_data = pd.read_excel('staff.xlsx')  # Replace 'path_to_staff.xlsx' with the actual path
types_data = pd.read_excel('types.xlsx')  # Replace 'path_to_types.xlsx' with the actual path

# Merge the data based on the 'Project' column
final_df = pd.merge(staff_data, types_data, on='Project')

# Display the first few rows of the merged DataFrame
print(final_df.head())


                     Job    Project      Month        Type
0              سائق لودر  Beni Suef 2015-10-01  Industrial
1             Storekeepe  Beni Suef 2015-10-01  Industrial
2  Survyour Section Head  Beni Suef 2015-10-01  Industrial
3                Foreman  Beni Suef 2015-10-01  Industrial
4                Foreman  Beni Suef 2015-10-01  Industrial


In [4]:
import pandas as pd

# Load the staff data from 'staff.xlsx'
staff_data = pd.read_excel('staff.xlsx')

# Display the first few rows to ensure the data is loaded correctly
print(staff_data.head())


                         Job    Project      Month
0                  سائق لودر  Beni Suef 2015-10-01
1              Site Engineer        Tb2 2015-10-01
2          Project Manager .        Tb2 2015-10-01
3              Site Engineer        Tb2 2015-10-01
4  Technical Office Engineer        Tb2 2015-10-01


In [5]:
# Convert 'Month' column to datetime if it's not already in datetime format
staff_data['Month'] = pd.to_datetime(staff_data['Month'])

# Group by 'Project' and 'Job' to calculate the peak number of employees and average start/end months
grouped_data = staff_data.groupby(['Project', 'Job']).agg(
    Peak_Number=('Job', 'count'),
    Average_Start_Month=('Month', 'min'),
    Average_End_Month=('Month', 'max')
).reset_index()

# Calculate the average peak number for each job across all projects
average_max_job_number = grouped_data['Peak_Number'].mean()

# Round down the average max job number to the nearest whole number
average_max_job_number = int(average_max_job_number)

# Display the results
print("Average max job number:", average_max_job_number)
print(grouped_data.head())


Average max job number: 20
                  Project                    Job  Peak_Number  \
0  10th of Ramadan Bridge  Blacksmith Supervisor            3   
1  10th of Ramadan Bridge   Carpenter Supervisor            2   
2  10th of Ramadan Bridge   Construction Foreman            3   
3  10th of Ramadan Bridge   Construction Manager            4   
4  10th of Ramadan Bridge    Document Controller            2   

  Average_Start_Month Average_End_Month  
0          2020-06-01        2020-08-01  
1          2020-06-01        2020-07-01  
2          2020-05-01        2020-07-01  
3          2020-05-01        2020-08-01  
4          2020-06-01        2020-07-01  


In [6]:
final_df = pd.merge(final_df, types_data, on='Project')

# Now 'final_df' should contain the 'Type' column
user_type = input("Enter the type: ")
filtered_df = final_df[final_df['Type'] == user_type]
# Proceed with calculating start and end months for the new deployment plan using filtered_df


# Calculate the start and end months for the new deployment plan:
number_of_months = int(input("Enter the number of months: "))  # Assuming user inputs the number of months

# Calculate start and end months
start_month = filtered_df['Start_Month'].mean()  # Get the average start month
end_month = start_month + number_of_months  # Calculate the end month

# Ensure the end month does not exceed the maximum end month in the dataset
max_end_month = filtered_df['End_Month'].max()
if end_month > max_end_month:
    end_month = max_end_month

#Create the new deployment plan:
new_deployment_plan = {
    'Type': user_type,
    'Start_Month': int(start_month),
    'End_Month': int(end_month),
    'Number_of_Jobs': filtered_df['Number_of_Jobs'].iloc[0]  # Assuming the number of jobs remains constant
}

print("New Deployment Plan:")
print(new_deployment_plan)


In [ ]:
# Read the Excel file
df = pd.read_excel("staff.xlsx")

# Read the project types
df_types = pd.read_excel("types.xlsx")

# Create dropdown menu options from unique project names
project_options = [{'label': project, 'value': project} for project in df['Project'].unique()]

# Create dropdown menu options for project types
type_options = [{'label': project_type, 'value': project_type} for project_type in df_types['Type'].unique()]

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [ ]:
# Define the layout of the app
app.layout = html.Div([
    html.H1("Deployment Plan"),
    dbc.Button("New Project", id="open", color="primary", className="mr-1"),
    dbc.Modal(
        [
            dbc.ModalHeader("New Project Details"),
            dbc.ModalBody(
                [
                    dcc.Input(id='type-input', type='text', placeholder='Type of Project'),
                    dcc.Input(id='duration-input', type='text', placeholder='Duration of Project')
                ]
            ),
            dbc.ModalFooter(
                [
                    dbc.Button("Close", id="close", className="ml-auto"),
                    dbc.Button("Save Changes", id="save-changes", className="ml-1")
                ]
            ),
        ],
        id="modal",
        size='lg',
        centered=True,
        backdrop='static'
    ),
    html.Div([
        html.Label('Type of Project:'),
        dcc.Dropdown(
            id='type-dropdown',
            options=type_options,
            value=type_options[0]['value'] if type_options else None  # Default value
        )
    ]),
    dcc.Dropdown(
        id='project-dropdown',
        options=project_options,
        value=project_options[0]['value']  # Default value
    ),
    dcc.Dropdown(
        id='job-dropdown',
        multi=True,
    ),
    dcc.Graph(id='deployment-plan'),
    html.Div(id='selected-project-type')  # Show the selected project type
])


In [ ]:
# Define callback to update job dropdown options based on selected project
@app.callback(
    Output('job-dropdown', 'options'),
    [Input('project-dropdown', 'value')]
)
def update_job_dropdown(selected_project):
    # Filter the DataFrame based on the selected project
    filtered_data = df[df['Project'] == selected_project]

    # Get unique job titles for the selected project
    job_options = [{'label': 'Select All', 'value': 'all'}] + [{'label': job, 'value': job} for job in
                                                                filtered_data['Job'].unique()]

    return job_options


In [ ]:
# Define callback to update the deployment plan based on selected project and job titles
@app.callback(
    Output('deployment-plan', 'figure'),
    [Input('project-dropdown', 'value'),
     Input('job-dropdown', 'value')]
)
def update_plan(selected_project, selected_jobs):
    if not selected_project or not selected_jobs:
        # Return an empty figure if no project or job is selected
        return {'data': [], 'layout': {}}

    if 'all' in selected_jobs:
        # If 'Select All' is selected, set selected_jobs to all job titles for the selected project
        filtered_data = df[df['Project'] == selected_project]
        selected_jobs = filtered_data['Job'].unique()

    # Filter the DataFrame based on the selected project and selected job titles
    filtered_data = df[(df['Project'] == selected_project) & (df['Job'].isin(selected_jobs))]

    if filtered_data.empty:
        # Return an empty figure if the filtered data is empty
        return {'data': [], 'layout': {}}

    # Get unique job titles for the selected project
    unique_jobs = filtered_data['Job'].unique()

    # Get unique months for the selected project and sort them
    months = sorted(filtered_data['Month'].unique())

    # Create a matrix to store staff count for each job title in each month
    staff_matrix = []

    # Iterate through each job title and count staff for each month
    for job in unique_jobs:
        job_data = filtered_data[filtered_data['Job'] == job]
        staff_count = [
            job_data[job_data['Month'] == month].shape[0] if not job_data[job_data['Month'] == month].empty else None
            for month in months
        ]
        staff_matrix.append(staff_count)

    # Get the maximum staff count for the selected project
    max_value = max([item for sublist in staff_matrix for item in sublist if item is not None])

    # Create data for the heatmap
    data = [
        go.Heatmap(
            z=staff_matrix,
            x=months,
            y=unique_jobs,
            colorscale='Viridis',
            showscale=True,
            zmin=1,  # Set minimum value for the color scale
            zmax=max_value,  # Set maximum value for the color scale
            zmid=0,  # Set midpoint value for the color scale
            colorbar=dict(tickmode='array', tickvals=list(range(1, max_value + 1)), ticktext=list(range(1, max_value + 1)))
        )
    ]

    # Define layout for the heatmap
    layout = go.Layout(
        title=f"Deployment Plan for {selected_project}",
        xaxis=dict(title='Months'),
        yaxis=dict(title='Job Titles'),
    )

    return {'data': data, 'layout': layout}


In [ ]:
# New callback for the modal popup
@app.callback(
    Output("modal", "is_open"),
    [Input("open", "n_clicks"), Input("close", "n_clicks"), Input("save-changes", "n_clicks")],
    [dash.dependencies.State("modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open


In [ ]:
# Callback to show the project type for the selected project
@app.callback(
    Output('selected-project-type', 'children'),
    [Input('project-dropdown', 'value')]
)
def display_project_type(selected_project):
    # Get the type of the selected project
    project_type = df_types[df_types['Project'] == selected_project]['Type'].values[0]
    return f'Type of Project: {project_type}'


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)